Install dropbox library

In [1]:
pip install dropbox

Note: you may need to restart the kernel to use updated packages.


Error processing line 3 of c:\Users\sang.huynh\Anaconda3\lib\site-packages\googleapis_common_protos-1.56.4-py3.10-nspkg.pth:

  Traceback (most recent call last):
    File "c:\Users\sang.huynh\Anaconda3\lib\site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 562, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored


### Get the ACCESS_TOKEN from DROPBOX DEVELOPERS for BUSINESS TEAM:

- Access https://www.dropbox.com/developers -> select App Console -> generate Access Token.
- Please notice that each Access Token only valids in 4 hours. We have to get the new Token after every 4 hours if needed.

In [2]:
user_name = 'xxx@domain.com'
pass_word = 'xxxx'
admin_id = 'dbmid:xxxxxx'
ACCESS_TOKEN = 'sl.xxxxx'

Authentication Types: https://www.dropbox.com/developers/reference/auth-types#admin-authentication

See this link to get API request/post format: https://www.dropbox.com/developers/reference/auth-types#user-authentication

### Step 1: List all Namespace ID in your Dropbox
Namespaces là bự nhất chứa các folders, để quét hết Dropbox cần chạy loop hết các Namespaces

In [10]:
### List all members in a Dropbox:
import requests
import pandas as pd
import os
# API endpoint URL to list members of the team
url = 'https://api.dropboxapi.com/2/team/namespaces/list'

# Headers containing the access token
headers = {
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}

# Make the API request
response = requests.post(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Print the response data
    namespace_list = response.json()
else:
    # Print error message if the request failed
    print(f"Error: {response.status_code} - {response.text}")


def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
    else:
        print(f"Folder '{folder_path}' already exists.")

# Specify the path of the folder you want to create
folder_path = "dropbox_test"

# Call the function to create the folder
create_folder_if_not_exists(folder_path)
namespace_path = f'{folder_path}/namespace_list.txt'

# Convert list to string for export file:
namespace_list_str = str(namespace_list)
# Open a file in write mode ('w')
with open(namespace_path, "w") as file:
    # Write the string to the file
    file.write(namespace_list_str)

print('number of namesapces in Dropbox: ', len(namespace_list['namespaces']))
print(namespace_list['namespaces'][1])

Folder 'dropbox_test' created successfully.
number of namesapces in Dropbox:  605
{'name': 'Sample Folder', 'namespace_id': '6906198080', 'namespace_type': {'.tag': 'shared_folder'}}


### Step 2: Access All Files and Folders in a Namespace:
Ở đây đang chạy cho namespace: Sample Folder

In [7]:
def get_all_files_folders(entries_lists):
    list_files = []
    list_folder = []
    parent_folder_name = entries_lists['parent_folder_name']
    for item in entries_lists['entries_lists']:        
        if item['.tag'] == 'folder':
            folder_name = item['name']
            folder_properties = {
            'folder_name': folder_name,
            'folder_path_name': f'{parent_folder_name}/{folder_name}',
            'folder_path_id': item['id']
            } 
            list_folder.append(folder_properties)
            # print(f'folder {parent_folder_name}/{folder_name} is added!')
        elif item['.tag'] == 'file':
            file_name = item['name']
            file_properties = {
            'file_name': file_name,
            'file_path':  f'{parent_folder_name}/',
            'file_id': item['id'],
            'client_modified': item['client_modified'],
            'server_modified': item['server_modified'],
            'size': item['size'],
            'is_downloadable': item['is_downloadable'],
            'content_hash': item['content_hash']
            }
            list_files.append(file_properties)
            # print(f'file {parent_folder_name}/{file_name} is added!')
    return(list_files, list_folder)

In [17]:

import json
# Tạo function đêt lấy list folders trong namespace:
def get_folder_lists_API(output_folder_path, parent_folder_path_id,parent_folder_path, parent_folder_name, admin_id,ACCESS_TOKEN):

    entries_lists = []
    no = 1
    # API endpoint URL to list members of the team
    url = 'https://api.dropboxapi.com/2/files/list_folder'
    # Headers containing the access token
    headers = {
        'Authorization': f'Bearer {ACCESS_TOKEN}',
        'Content-Type': 'application/json',
        "Dropbox-API-Select-Admin": admin_id
        # 'Dropbox-API-path_Root': {".tag": "root", "root": namespace_id}
    }
   
    # Body:
    data = {
    "path": parent_folder_path_id,
    "recursive": True
    }

    # Make the API request
    response = requests.post(url, headers=headers, data=json.dumps(data))

    # Check if the request was successful
    if response.status_code == 200:
        # Print the response data
        lists_df = response.json()
        entries_lists.extend(lists_df['entries'])
        # print(len(entries_lists), no)
    else:
        # Print error message if the request failed
        print(f"Error: {response.status_code} - {response.text}")
    
    while lists_df['has_more']:
        no = no + 1
        cursor = lists_df['cursor']
        url = 'https://api.dropboxapi.com/2/files/list_folder/continue'

        # Headers containing the access token
        headers = {
            'Authorization': f'Bearer {ACCESS_TOKEN}',
            'Content-Type': 'application/json',
            "Dropbox-API-Select-Admin": admin_id
            # 'Dropbox-API-path_Root': {".tag": "root", "root": namespace_id}
        }
        # Body:
        data = {
        "cursor": cursor
        }

        # Make the API request
        response = requests.post(url, headers=headers, data=json.dumps(data))

        # Check if the request was successful
        if response.status_code == 200:
            # Print the response data
            lists_df = response.json()
            entries_lists.extend(lists_df['entries'])
            # print(len(entries_lists), no)

        else:
            # Print error message if the request failed
            print(f"Error: {response.status_code} - {response.text}")        
    
    parent_entries = {  "parent_folder_name": parent_folder_name,
                        "parent_folder_path": parent_folder_path,
                        "entries_lists": entries_lists }
    
    # Call the function to create the folder
    create_folder_if_not_exists(output_folder_path)

    file_id = parent_folder_path_id.split(':')[1]
    parent_entries_path = f'{output_folder_path}/parent_entries_{file_id}.txt'
    parent_entries_str = str(parent_entries)
    # Open a file in write mode ('w')
    with open(parent_entries_path, "w") as file:
        # Write the string to the file
        file.write(parent_entries_str)
    
    print(parent_folder_path, 'number of objects: ', len(entries_lists))


    return(parent_entries)

# chạy cho namespace 03 Current Deal:
namespace = namespace_list['namespaces'][4]
namespace_id = namespace['namespace_id']
parent_folder_path_id = f'ns:{namespace_id}'
parent_folder_name = namespace['name']
parent_folder_path = f'{parent_folder_name}/'
output_folder_path = f'dropbox_test/{parent_folder_name}'
namespace_folders_list = get_folder_lists_API(output_folder_path, parent_folder_path_id,parent_folder_path, parent_folder_name, admin_id,ACCESS_TOKEN)


Folder 'dropbox_test/03 Current Deals' already exists.
03 Current Deals/ number of objects:  4344


### Step 3: Get all files in all folders in a namespace
Using recursive function to get data with infinited loops

In [11]:
def get_all_files_folders(entries_lists):
    list_files = []
    list_folder = []
    parent_folder_name = entries_lists['parent_folder_name']
    for item in entries_lists['entries_lists']:        
        if item['.tag'] == 'folder':
            folder_name = item['name']
            folder_properties = {
            'folder_name': folder_name,
            'folder_path_name': f'{parent_folder_name}/{folder_name}',
            'folder_path_id': item['id']
            } 
            list_folder.append(folder_properties)
            print(f'folder {parent_folder_name}/{folder_name} is added!')
        elif item['.tag'] == 'file':
            file_name = item['name']
            file_properties = {
            'file_name': file_name,
            'file_path':  f'{parent_folder_name}/',
            'file_id': item['id'],
            'client_modified': item['client_modified'],
            'server_modified': item['server_modified'],
            'size': item['size'],
            'is_downloadable': item['is_downloadable'],
            'content_hash': item['content_hash']
            }
            list_files.append(file_properties)
            print(f'file {parent_folder_name}/{file_name} is added!')
    return(list_files, list_folder)
# list_files, list_folder = get_all_files_folders(namespace_folders_list)

### Step 5: Extract modified files from the file list

In [ ]:
# generate dataframe of file list:   
columns = list(list_files[0].keys())
list_files_df = pd.DataFrame(list_files, columns=columns)
list_files_df.head()

In [ ]:
import datetime
last_migration_time = datetime.datetime.now()
list_files_df[list_files_df['client_modified'] >= last_migration_time or \
              list_files_df['server_modified'] >= last_migration_time]

Download files from Dropbox

In [40]:
import os

def download_files_Dropbox(file_id, folder_path):
    url = 'https://content.dropboxapi.com/2/files/download'

    # Define the folder where you want to save the file
    folder_path = './downloaded_folder/'

    # Create the folder if it doesn't exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
    file_path = os.path.join(folder_path, os.path.basename(file_name))

    path_id = '"\"path\": \"' +   file_id + '\"}"'

    # Headers containing the access token
    headers = {
        'Authorization': f'Bearer {ACCESS_TOKEN}',
        "Dropbox-API-Select-Admin": admin_id,
        'Dropbox-API-Arg': path_id
    }


    # Make the API request
    response = requests.post(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Write the downloaded content to a file
        with open(f'{file_path}', 'wb') as f:
            f.write(response.content)
        print(f"{file_name} downloaded successfully.")
    else:
        # Print error message if download failed
        print("Error:", response.status_code, response.text)


file_id = list_files_namespace[43]['file_id']
file_name = list_files_namespace[43]['file_name']
download_files_Dropbox(file_id, folder_path)

14. The Home Staging Invoice 341.pdf downloaded successfully.
